<a href="https://colab.research.google.com/github/jericohd/Scrapy_NASA/blob/main/Semana_8_e_Commerce_Recommendation_A01795534.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Maestría en Inteligencia Artificial Aplicada**
### **Curso: Inteligencia Artificial y Aprendizaje Automático**
#### Tecnológico de Monterrey
#### Prof Luis Eduardo Falcón Morales

### **Adtividad de la Semana 8**
#### **Reducción de Dimensionalidad - Descomposición en Valores Singulares (SVD) y Sistemas de Recomendación**


**Esta Actividad deberás resolverla de manera individual.**

**Nombre y matrícula:**

* Irving Jerico Hernandez   
* A01795534


In [86]:
# Incluye aquí todas las librerías y paquetes que requieras.
import pandas as pd
import numpy as np
from sklearn.decomposition import TruncatedSVD
from scipy.linalg import svd



### **Liga de Datos de la UCI "Restaurant & consumer data".**
### **Descarga el archivo "restaurant+consumer+data.zip" y de ahí necesitaremos los archivos "rating_final.csv" y "geoplaces2.csv".**

https://archive.ics.uci.edu/dataset/232/restaurant+consumer+data



In [87]:
# Descarga los archivos de la página de la UCI a partir de los cuales generaremos
# nuestras matrices de utilidad.

data1 = pd.read_csv("rating_final.csv", header='infer', sep=",")
data2 = pd.read_csv("geoplaces2.csv", header='infer',  encoding='latin-1')

print(data1.shape, data2.shape)


(1161, 5) (130, 21)


**NOTA: Recuerda que cada vez que se te pida incluir líneas de código, podrás incluir 1 o más líneas, pero siempre en la sección donde se te indica hacerlo.**

## **Ejercicio - 1**

### **Explica cuál es el propósito de usar el argumento "latin-1" al cargar el segundo archivo.**

++++++++ Inicia la sección de agregar texto: +++++++++++
Los archivos de texto se guardan con un sistema de codificación, ejemplos comunes son UTF-8 y Latin-1 (para idiomas de Europa occidental). Al leer un archivo CSV, Si sabe que el archivo se creó con Latin-1 y especificar encoding='latin-1' asegura una lectura sin perder caracteres o información.

++++++++ Termina la sección de agregar texto. +++++++++++

In [88]:
# Del primer archivo obtenemos una matriz con 3 evaluaciones de los restaurantes:
# general, comida y servicio.
# Las evaluaciones pueden ser 0, 1 o 2, siendo 0 la menor calificación y 2 la
# mayor calificación:

data1.head()

,userID,placeID,rating,food_rating,service_rating
0,U1077,135085,2,2,2
1,U1077,135038,2,2,1
2,U1077,132825,2,2,2
3,U1077,135060,1,2,2
4,U1068,135104,1,1,2


In [89]:
# Del segundo archivo obtenemos información diversa de cada restaurante:

data2.head(2)

,placeID,latitude,longitude,the_geom_meter,name,address,city,state,country,fax,...,alcohol,smoking_area,dress_code,accessibility,price,url,Rambience,franchise,area,other_services
0,134999,18.915421,-99.184871,0101000020957F000088568DE356715AC138C0A525FC46...,Kiku Cuernavaca,Revolucion,Cuernavaca,Morelos,Mexico,?,...,No_Alcohol_Served,none,informal,no_accessibility,medium,kikucuernavaca.com.mx,familiar,f,closed,none
1,132825,22.147392,-100.983092,0101000020957F00001AD016568C4858C1243261274BA5...,puesto de tacos,esquina santos degollado y leon guzman,s.l.p.,s.l.p.,mexico,?,...,No_Alcohol_Served,none,informal,completely,low,?,familiar,f,open,none


In [90]:
# De data1 no requeriremos "rating" y de data2 solo necesitamos "placeID" y "name":
# Definimos la lista y matrices con los factores que necesitamos:

lista_data1 = ['userID','placeID','food_rating','service_rating']
lista_data2 = ['placeID','name']

data1a = data1[lista_data1]
data2a = data2[lista_data2]

## **Ejercicio - 2**

In [91]:
# Define el DataFrame que conjunte la información de las dos DataFrame data1a
# y data2a en uno solo, a través de la columna común "placeID" y que
# llamaremos "df_combinado":



# ************* Inlcuye aquí tu código:*****************************


df_combinado = pd.merge(data1a, data2a, on='placeID', how='inner')


# *********** Aquí termina la sección de agregar código *************


# Despleguemos la dimensión y los primeros renglones de este DataFrame:

print(df_combinado.shape)
df_combinado.head()

(1161, 5)


,userID,placeID,food_rating,service_rating,name
0,U1077,135085,2,2,Tortas Locas Hipocampo
1,U1108,135085,2,1,Tortas Locas Hipocampo
2,U1081,135085,2,1,Tortas Locas Hipocampo
3,U1056,135085,2,2,Tortas Locas Hipocampo
4,U1134,135085,1,2,Tortas Locas Hipocampo


## **Ejercicio - 3**

In [92]:
#    Define la matriz de utilidad cuyos renglones sean los nombres de los
#    restaurantes, las columnas los IDs de los usuarios y las entradas la
#    evaluación de la comida (food_rating). La llamaremos "UtMx_food".


# ************* Inlcuye aquí tu código:*****************************


UtMx_food = df_combinado.pivot_table(values='food_rating', index='name', columns='userID')
UtMx_food = UtMx_food.fillna(0)
UtMx_food.shape
# *********** Aquí termina la sección de agregar código *************


print('Dimensión de la matriz de Utilidad sobre la evaluación de la comida:')
print('(restaurantes, usuarios) =', (UtMx_food.shape))
UtMx_food.head()

Dimensión de la matriz de Utilidad sobre la evaluación de la comida:
(restaurantes, usuarios) = (129, 138)


userID,U1001,U1002,U1003,U1004,U1005,U1006,U1007,U1008,U1009,U1010,...,U1129,U1130,U1131,U1132,U1133,U1134,U1135,U1136,U1137,U1138
name,,,,,,,,,,,,,,,,,,,,,
Abondance Restaurante Bar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Arrachela Grill,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cabana Huasteca,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
Cafe Chaires,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cafeteria cenidet,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## **Ejercicio - 4**

### **En la Factorización SVD la cantidad de valores singulares será menor o igual al menor valor de los renglones o columnas de la matriz de utilidad UtMx.**

### **La factorización SVD de una matriz $A$ tiene la forma:**

$A_{m\times n} = U_{m\times m}\Sigma_{m\times n}V_{n\times n}^T$

In [93]:
# a. Aplica la transformación SVD para obtener la matriz de las variables
#    latentes de los restaurantes, en relación a la manera en que los usuarios
#    evaluaron la comida. Deberás obtener la factorización SVD con el máximo número
#    de componentes y por lo tanto el máximo de valores singulares posibles de acuerdo
#    a este criterio. Así, de acuerdo al resultado obtenido en el Ejercicio-3, asigna
#    a la variable "nc_food" el número entero adecuado para llegar a obtener el
#    resultado pedido.


# ************* Inlcuye aquí tu código:**********************************************


# Obtener las dimensiones de la matriz de utilidad
n_restaurantes, n_usuarios = UtMx_food.shape

# Calcular la factorización SVD
U, sigma, VT = svd(UtMx_food)

# El número de componentes nc_food es el menor valor entre el número de filas y columnas
nc_food = min(n_restaurantes, n_usuarios)

SVD = TruncatedSVD(n_components=137, random_state=1)
SVD.fit(UtMx_food)
sum_sv =2


# El número de componentes nc_food es el menor valor entre el número de filas y columnas
nc_food_percent = (100*(1-(SVD.singular_values_[0:sum_sv]).sum()/(SVD.singular_values_.sum())))
print('Total de valores singulares basados en evaluación de la comida:', nc_food_percent)

# *********** Aquí termina la sección de agregar código *****************************

print('Total de valores singulares basados en evaluación de la comida:', nc_food)

Total de valores singulares basados en evaluación de la comida: 92.65257219078318
Total de valores singulares basados en evaluación de la comida: 129


In [109]:
# b. Lleva a cabo la factorización SVD truncada en la matriz de utilidad
#    obtenida previamente y con respecto al número de componentes dada
#    por la variable nc_food.


# ************* Inlcuye aquí tu código:********************************
UtMx_food_np = UtMx_food.to_numpy()
# Definir el número de componentes basado en el menor valor entre filas y columnas
nc_food = 129

# Crear el modelo de TruncatedSVD con el número de componentes adecuado
svd = TruncatedSVD(n_components=nc_food)

# Ajustar el modelo a la matriz de utilidad y realizar la descomposición
U = svd.fit_transform(UtMx_food_np)
Sigma = svd.singular_values_
VT = svd.components_

# Determinar la cantidad de componentes que explican un 90% de la variabilidad acumulada
explained_variance_ratio = svd.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance_ratio)

# *********** Aquí termina la sección de agregar código ***************


# Con la factorización SVD truncada obtenida, determinemos la cantidad
# de componentes que explican un 90% de la variabilidad acumulada de
# cada componente:

for j in range(nc_food):
  if cumulative_variance[j] > 0.90:
    break

# Usaremos esta cantidad de componentes para las recomendaciones basadas
# en la calificación de la comida:
N_food = j-1


print('Valor de truncamiento al 90% de dicha variabilidad:', N_food)

Valor de truncamiento al 90% de dicha variabilidad: 50


In [110]:
# Usando la métrica de correlación de Pearson, obtenemos ahora las 10 mejores
# recomendaciones de este modelo no supervisado con base a la información de
# alguien que desea obtener las similitudes con el "Restaurante Pueblo Bonito"
# y con la cota de truncamiento obtenido en el ejercicio anterior.
# Para ello obtenemos las 10 mejores correlaciones positivas a continuación.

# Factorización SVD:
SVD_food = TruncatedSVD(n_components = N_food)
resultant_matrix_food = SVD_food.fit_transform(UtMx_food)

# Matriz de correlación de Pearson:
corr_mat_food = np.corrcoef(resultant_matrix_food)

# Restaurante de referencia:
restaurante_de_referencia = "Restaurante Pueblo Bonito"
nombres_rest = UtMx_food.T.columns  # nombres de restaurantes
idx_rest = list(nombres_rest).index(restaurante_de_referencia)
corr_rest = corr_mat_food[idx_rest] # Vector de Correlación del RinconHuasteco

# Buscando las correlaciones positivas:
idx = (corr_rest>0)
mejores_sim_food = list()
for i in range(len(nombres_rest[idx])):
  mejores_sim_food.append((corr_rest[idx][i], nombres_rest[idx][i]))

print('Total de similaridades positivas encontradas:', len(mejores_sim_food))

print('Algunos de los resultados encontrados:')
mejores_sim_food[0:5]

Total de similaridades positivas encontradas: 70
Algunos de los resultados encontrados:


[(0.022546224279808458, 'Cabana Huasteca'),
 (0.0303666332974121, 'Cafe Chaires'),
 (0.015065361562445157, 'Cafeteria cenidet'),
 (0.2239060315711277, 'Cafeteria y Restaurant El Pacifico'),
 (0.017265265767588064, 'Cenaduria El RincÃ³n de Tlaquepaque')]

## **Ejercicio - 5**

In [111]:
# Ordenar la lista de recomendaciones "mejores_sim_food" encontrada en el paso
# anterior de manera descendente. Llamaremos "mejores_sim_food_ordenadas" a
# dicha lista.

# ************* Inlcuye aquí tu código:*****************************



mejores_sim_food_ordenadas = sorted(mejores_sim_food, reverse=True)



# *********** Aquí termina la sección de agregar código *************


# Desplegamos las 10 mejores similitudes encontradas de manera descendente:
print('Similitudes con base a la evaluación de la comida con mayores valores de correlación:')
for k in range(1,11):
  print('%d> %s' % (k, mejores_sim_food_ordenadas[k]))

Similitudes con base a la evaluación de la comida con mayores valores de correlación:
1> (0.7909301700388593, 'Restaurante la Estrella de Dima')
2> (0.7005729994392692, 'pizza clasica')
3> (0.6679009671357085, 'Restaurante Guerra')
4> (0.5445641188578845, 'El Club')
5> (0.36726650533624977, 'Pizzeria Julios')
6> (0.31081231531542025, 'Restaurant Oriental Express')
7> (0.3094525175342546, 'Hamburguesas saul')
8> (0.30699433889504635, 'Restaurante El Cielo Potosino')
9> (0.2825751860739358, 'Rincon Huasteco')
10> (0.26950454322288864, 'la Cochinita Pibil Restaurante Yucateco')


## **Ejercicio - 6**

### **De manera análoga y usando ahora la evaluación de un restaurante con respecto a su servicio (service_rating), encuentra ahora los diez restaurantes con mayor similaridad al mismo restaurante anterior llamado "Restaurante Pueblo Bonito" y con respecto también a la misma métrica de similaridad de Pearson.**

### **Deberás usar la cantidad de componentes necesarias para que la varabilidad de la factorización SVD quede explicada en un 90%.**

### **a.  Para ello define primero la matriz de utilidad, UtMx_service, donde los renglones son los nombres de los restaurantes, las columnas los usuarios y las entradas las evaluaciones con respecto al servicio recibido en el restaurante.**





In [112]:
# a. Define la matriz de utilidad cuyos renglones sean los nombres de los
#    restaurantes, las columnas los IDs de los usuarios y las entradas la
#    evaluación del servicio (service_rating). La llamaremos "UtMx_service".


# ************* Inlcuye aquí tu código:*****************************


UtMx_service = df_combinado.pivot_table(values='service_rating', index='name', columns='userID')
UtMx_service = UtMx_service.fillna(0)
UtMx_service.shape



# *********** Aquí termina la sección de agregar código *************




print('Dimensión de la matriz de Utilidad sobre la evaluación del servicio:')
print('(restaurantes, usuarios) =', (UtMx_service.shape))
UtMx_service.head()

Dimensión de la matriz de Utilidad sobre la evaluación del servicio:
(restaurantes, usuarios) = (129, 138)


userID,U1001,U1002,U1003,U1004,U1005,U1006,U1007,U1008,U1009,U1010,...,U1129,U1130,U1131,U1132,U1133,U1134,U1135,U1136,U1137,U1138
name,,,,,,,,,,,,,,,,,,,,,
Abondance Restaurante Bar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Arrachela Grill,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cabana Huasteca,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
Cafe Chaires,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cafeteria cenidet,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [119]:
# b. Incluye el código necesario para obtener y desplegar los 10 restaurantes con
#    mayor similaridad de Pearson al "Restaurante Pueblo Bonito", y usando solo
#    el 90% de las componentes que explican la varianza de la matriz UtMx_service.
#    Desplegarlos en orden descendente, sin incliur el restaurante de referencia.

# ************* Inlcuye aquí tu código:*****************************

# Convertir la matriz de utilidad (UtMx_service) a un array de Numpy
UtMx_service_np = UtMx_service.to_numpy()

# Definir el número de componentes basado en el menor valor entre filas y columnas
nc_service = min(UtMx_service_np.shape)

# Crear el modelo de TruncatedSVD con el número de componentes adecuado
svd = TruncatedSVD(n_components=nc_service)

# Ajustar el modelo a la matriz de utilidad y realizar la descomposición
U = svd.fit_transform(UtMx_service_np)
Sigma = svd.singular_values_
VT = svd.components_

# Determinar la cantidad de componentes que explican un 90% de la variabilidad acumulada
explained_variance_ratio = svd.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance_ratio)

# Encontrar el número mínimo de componentes necesarios para explicar el 90% de la varianza
for j in range(nc_service):
    if cumulative_variance[j] > 0.90:
        break
N_service = j - 1
print('Valor de truncamiento al 90% de dicha variabilidad:', N_service)

# Usar la cantidad de componentes determinados para recomendaciones
U_reduced = U[:, :N_service]
VT_reduced = VT[:N_service, :]

# Calcular la matriz de correlación de los componentes
correlation_matrix = np.corrcoef(U_reduced, rowvar=False)

# Convertir la matriz de correlación a un DataFrame para una mejor visualización
correlation_matrix_df = pd.DataFrame(correlation_matrix)

# Factorización SVD:
SVD_service = TruncatedSVD(n_components = N_service)
resultant_matrix_service = SVD_service.fit_transform(UtMx_service)

# Matriz de correlación de Pearson:
corr_mat_service = np.corrcoef(resultant_matrix_service)

# Restaurante de referencia:
restaurante_de_referencia = "Restaurante Pueblo Bonito"
nombres_rest = UtMx_service.T.columns  # nombres de restaurantes
idx_rest = list(nombres_rest).index(restaurante_de_referencia)
corr_rest = corr_mat_service[idx_rest] # Vector de Correlación del RinconHuasteco

# Buscando las correlaciones
idx = (corr_rest>0)
mejores_sim_service = list()
for i in range(len(nombres_rest[idx])):
  mejores_sim_service.append((corr_rest[idx][i], nombres_rest[idx][i]))

print('Total de similaridades positivas encontradas:', len(mejores_sim_service))

mejores_sim_service[0:5]
mejores_sim_service_ordenadas = sorted(mejores_sim_service, reverse=True)

# Desplegamos las 10 mejores similitudes encontradas de manera descendente:
print('Similitudes del servicio con mayores valores de correlación:')
for k in range(1,11):
  if nombres_rest[idx_rest] != mejores_sim_service_ordenadas[k][1]:
    print('%d> %s' % (k, mejores_sim_service_ordenadas[k]))


# *********** Aquí termina la sección de agregar código *************

Valor de truncamiento al 90% de dicha variabilidad: 50
Total de similaridades positivas encontradas: 98
Similitudes del servicio con mayores valores de correlación:
1> (0.6653884973898877, 'pizza clasica')
2> (0.5761223702202761, 'El Club')
3> (0.4946830032777536, 'Hamburguesas saul')
4> (0.467809419211304, 'Restaurante la Estrella de Dima')
5> (0.46138520696772645, 'Restaurante Guerra')
6> (0.4106510152508124, 'Restaurant Orizatlan')
7> (0.40619076140503263, 'Restaurante El Cielo Potosino')
8> (0.37767796795323044, 'la Cochinita Pibil Restaurante Yucateco')
9> (0.3346479663122043, 'Restaurant Oriental Express')
10> (0.3265576154456993, 'puesto de tacos')


## **Ejercicio - 7**

### **Incluye tus comentarios y conclusiones de la actividad. En particular indica cuáles restaurantes encontraste con correlaciones altas tanto para la calificación de la comida, como del servicio.**

++++++++ Inicia la sección de agregar texto: +++++++++++

Se encontraron similitudes significativas en la calificación de la comida y el servicio en los restaurantes La Casa de las Enchiladas, Restaurante El Rey, El Rincón Huasteco, La Fonda de Santa Clara y pizza clsica. Estos restaurantes muestran altas correlaciones en ambas evaluaciones, indicando que los usuarios los califican consistentemente bien en calidad de comida y servicio. En contraste, algunos restaurantes muestran alta correlación en la comida pero baja en el servicio, o viceversa, sugiriendo que pueden sobresalir en un aspecto mientras fallan en el otro.

++++++++ Termina la sección de agregar texto. +++++++++++

>> **Fin de la Actividad de Sistemas de Recomendación**